In [1]:
# Importing librarys
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

import csv
from matplotlib import pyplot as plt

import librosa
import os

from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# Ներկայացնում եմ առաջադրանքի կատարումը։ Օգտագործել եմ տարբեր մեթոդներ, նպատակը՝ գտնել ամենա հարմարը հիպերպարամետրերը փորձարկելու և մոդելը ձևափոխելու համար։ Ցավոք, պատերազմական դրության պատճառով մինչև դեդլայնը շատ քիչ ժամանակ է հնարավոր եղել տրամադրել, գումարած, ամեն պարապմունքը մեծ չափանի դատայի վրա իմ մոտ տևում է ժամեր, այդ իսկ պատճառով հիպերպարամետրերը չեմ հասցրել փոփոխել, շերտերի անհրաժեշտ քանակը և կազմությունը հնարավոր չէր որոշել։

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_virtual_device_configuration(
                            gpus[0],
                            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# First and simple method

df = pd.read_csv('train_curated'+'.csv', encoding='utf-8')
target = df.pop('labels')

tokenizer = tfds.features.text.Tokenizer()

label_names = []
for row in list(set(target.values)):
    label_names.extend(tokenizer.tokenize(row))
label_names = list(set(label_names)) # list of labels

train_data = tf.data.Dataset.from_tensor_slices(('train_curated/'+df.values, target.values))

def DataPreproc(filename_tensor, labels):
    
    text = filename_tensor.numpy()[0]
    labels = labels.numpy()
    y, sr = librosa.load(text, mono = True)

    if y.shape[0] < sr:
        y = np.pad(y, (0, sr-y.shape[0]), constant_values=(0, -1))
    
    # Extracting features
    fet = [librosa.feature.chroma_stft(y=y, sr=sr), 
           librosa.feature.spectral_centroid(y=y, sr=sr),
           librosa.feature.spectral_bandwidth(y=y, sr=sr),
           librosa.feature.spectral_rolloff(y=y, sr=sr),
           librosa.feature.zero_crossing_rate(y),
           librosa.feature.mfcc(y=y, sr=sr)]
    
    #Preprocessing feature vectors and matrixs to matrixs
    features = fet[0]
    for i in fet[1:]:
        features = np.append(features, i, axis=0)
    features = features.T
    
    # Text labels to int vector
    labels = tokenizer.tokenize(labels)
    label = np.zeros_like(label_names, dtype=np.int32)
    for lb in labels:
        label[label_names.index(lb)] = 1
        
    return features, label

def DataPreproc_map(filename_tensor, labels):
    return tf.py_function(DataPreproc, inp = [filename_tensor, labels],
                          Tout = (tf.float32, tf.int32))

EPOCHS = 10 # Epochs for training
BATCH_SIZE = 30 # Batch size of training and validation data
BUFFER = 3000 # Buffer for shuffling

#train_data = train_data.shuffle(BUFFER, reshuffle_each_iteration=False)

train_data = train_data.map(DataPreproc_map)

test_data = train_data.skip(3200)
valid_data = test_data.take(1000)
test_data = test_data.skip(1000)
train_data = train_data.take(3200)

for ex in train_data.take(3):
    print(ex[0].shape)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None, None], [None]))
valid_data = valid_data.padded_batch(BATCH_SIZE, padded_shapes=([None, None], [None]))
test_data = test_data.batch(1)


(304, 36)
(89, 36)
(66, 36)


In [7]:
for ex in train_data.take(1):
    shape_1 = ex[0].shape[2]
    output_shape = ex[1].shape[1]

# Building LSTM model
# LSTM layer - in bidirectional, for first - (1,64)

bi_lstm_model_1 = tf.keras.Sequential([
    layers.Bidirectional(layers.LSTM(512, name = 'lstm-1'), input_shape=(None, shape_1),
                                  name = 'bidir-lstm'),
    #layers.Bidirectional(layers.LSTM(64, name = 'lstm-2'),
     #                             name = 'bidir-lstm_2'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    #tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.1),
    
    tf.keras.layers.Dense(output_shape, activation = 'sigmoid')
])

bi_lstm_model_1.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidir-lstm (Bidirectional)   (None, 128)               51712     
_________________________________________________________________
dense_3 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 86)                44118     
Total params: 424,534
Trainable params: 424,534
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Compile and train the model
# Use Adam optimizer, regularization parameter - None (for first), loss function 
#- binarycrossentropy, for cumputing loss for each label more effectively

bi_lstm_model_1.compile(optimizer = tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics = ['accuracy'])
history = bi_lstm_model_1.fit(train_data, 
                            validation_data = valid_data,
                            epochs = 20)


Epoch 1/20
  1/107 [..............................] - ETA: 0s - loss: 0.6891 - accuracy: 0.0333

D:\Anaconda\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


107/107 [==============================] - 2833s 26s/step - loss: 0.1141 - accuracy: 0.0134 - val_loss: 0.0761 - val_accuracy: 0.0130
Epoch 2/20
107/107 [==============================] - 2810s 26s/step - loss: 0.0746 - accuracy: 0.0434 - val_loss: 0.0742 - val_accuracy: 0.0320
Epoch 3/20
107/107 [==============================] - 2887s 27s/step - loss: 0.0724 - accuracy: 0.0572 - val_loss: 0.0727 - val_accuracy: 0.0430
Epoch 4/20
107/107 [==============================] - ETA: 0s - loss: 0.0711 - accuracy: 0.0728 

In [ ]:
# Evaluate on test data

test_results = bi_lstm_model_1.evaluate(test_data)
print('Test acc.: {:.2f}%'.format(test_results[1]*100))

In [19]:
# Model 1_2 version

# First and simple method

df = pd.read_csv('train_curated'+'.csv', encoding='utf-8')
target = df.pop('labels')

tokenizer = tfds.features.text.Tokenizer()

label_names = []
for row in list(set(target.values)):
    label_names.extend(tokenizer.tokenize(row))
label_names = list(set(label_names)) # list of labels

train_data = tf.data.Dataset.from_tensor_slices(('train_curated/'+df.values, target.values))

def DataPreproc(filename_tensor, labels):
    
    text = filename_tensor.numpy()[0]
    labels = labels.numpy()
    y, sr = librosa.load(text, mono = True)

    if y.shape[0] < sr:
        y = np.pad(y, (0, sr-y.shape[0]), constant_values=(0, -1))
    
    # Extracting features
    fet = [librosa.feature.chroma_stft(y=y, sr=sr), 
           librosa.feature.spectral_centroid(y=y, sr=sr),
           librosa.feature.spectral_bandwidth(y=y, sr=sr),
           librosa.feature.spectral_rolloff(y=y, sr=sr),
           librosa.feature.zero_crossing_rate(y),
           librosa.feature.mfcc(y=y, sr=sr)]
    
    #Preprocessing feature vectors and matrixs to matrixs
    features = fet[0]
    for i in fet[1:]:
        features = np.append(features, i, axis=0)
    features = features.T
    
    # Text labels to int vector
    labels = tokenizer.tokenize(labels)
    label = np.zeros_like(label_names, dtype=np.int32)
    for lb in labels:
        label[label_names.index(lb)] = 1
        
    return features, label

def DataPreproc_map(filename_tensor, labels):
    return tf.py_function(DataPreproc, inp = [filename_tensor, labels],
                          Tout = (tf.float32, tf.int32))

EPOCHS = 5 # Epochs for training
BATCH_SIZE = 30 # Batch size of training and validation data
BUFFER = 3000 # Buffer for shuffling

#train_data = train_data.shuffle(BUFFER, reshuffle_each_iteration=False)

train_data = train_data.map(DataPreproc_map)

test_data = train_data.skip(3200)
valid_data = test_data.take(1000)
test_data = test_data.skip(1000)
train_data = train_data.take(3200)

for ex in train_data.take(3):
    print(ex[0].shape)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None, None], [None]))
valid_data = valid_data.padded_batch(BATCH_SIZE, padded_shapes=([None, None], [None]))
test_data = test_data.batch(1)

for ex in train_data.take(1):
    shape_1 = ex[0].shape[2]
    output_shape = ex[1].shape[1]

# Building LSTM model
# LSTM layer - in bidirectional, for first - (1,64)

bi_lstm_model_1 = tf.keras.Sequential([
    layers.Bidirectional(layers.LSTM(128, name = 'lstm-1'), input_shape=(None, shape_1),
                                  name = 'bidir-lstm'),
    #layers.Bidirectional(layers.LSTM(64, name = 'lstm-2'),
     #                             name = 'bidir-lstm_2'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(512, activation = 'relu'),
    #tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.1),
    
    tf.keras.layers.Dense(output_shape, activation = 'sigmoid')
])

bi_lstm_model_1.summary()

#Compile and train the model
# Use Adam optimizer, regularization parameter - None (for first), loss function 
#- binarycrossentropy, for cumputing loss for each label more effectively

bi_lstm_model_1.compile(optimizer = tf.keras.optimizers.Adam(),
                      loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
                      metrics = [tf.keras.metrics.BinaryAccuracy(threshold=0.4), 'accuracy'])
history = bi_lstm_model_1.fit(train_data, 
                              validation_data = valid_data,
                              shuffle=True,
                              epochs = EPOCHS)


(304, 36)
(89, 36)
(66, 36)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidir-lstm (Bidirectional)   (None, 256)               168960    
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 86)                44118     
Total params: 607,318
Trainable params: 607,318
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
  1/107 [..............................] - ETA: 0s - loss: 0.6908 - binary_acc

D:\Anaconda\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


107/107 [==============================] - 2980s 28s/step - loss: 0.1102 - binary_accuracy: 0.9564 - accuracy: 0.0247 - val_loss: 0.0753 - val_binary_accuracy: 0.9852 - val_accuracy: 0.0510
Epoch 2/5
107/107 [==============================] - 3109s 29s/step - loss: 0.0733 - binary_accuracy: 0.9855 - accuracy: 0.0566 - val_loss: 0.0722 - val_binary_accuracy: 0.9852 - val_accuracy: 0.0840
Epoch 3/5
107/107 [==============================] - 3315s 31s/step - loss: 0.0711 - binary_accuracy: 0.9855 - accuracy: 0.0744 - val_loss: 0.0711 - val_binary_accuracy: 0.9852 - val_accuracy: 0.0860
Epoch 4/5
107/107 [==============================] - 2998s 28s/step - loss: 0.0694 - binary_accuracy: 0.9855 - accuracy: 0.0831 - val_loss: 0.0703 - val_binary_accuracy: 0.9852 - val_accuracy: 0.0960
Epoch 5/5
107/107 [==============================] - 3434s 32s/step - loss: 0.0681 - binary_accuracy: 0.9855 - accuracy: 0.0934 - val_loss: 0.0697 - val_binary_accuracy: 0.9852 - val_accuracy: 0.0990


In [39]:
# Second and simple method

df = pd.read_csv('train_curated'+'.csv', encoding='utf-8')
target = df.pop('labels')

tokenizer = tfds.features.text.Tokenizer()

label_names = []
for row in list(set(target.values)):
    label_names.extend(tokenizer.tokenize(row))
label_names = list(set(label_names))

train_data = tf.data.Dataset.from_tensor_slices(('train_curated/'+df.values, target.values))

def DataPreproc(filename_tensor, labels):
    text = filename_tensor.numpy()[0]
    labels = labels.numpy()
    y, sr = librosa.load(text, mono = True)

    if y.shape[0] == 0:
        y = np.zeros((sr), dtype=np.float32)
    y.resize((int(y.shape[0]/sr), sr), refcheck=False)
    
    labels = tokenizer.tokenize(labels)
    label = np.zeros_like(label_names, dtype=np.int32)
    for lb in labels:
        label[label_names.index(lb)] = 1
        
    return y, label
def DataPreproc_map(filename_tensor, labels):
    return tf.py_function(DataPreproc, inp = [filename_tensor, labels],
                          Tout = (tf.float32, tf.int32))

BATCH_SIZE = 30
BUFFER = 3000

#train_data = train_data.shuffle(BUFFER, reshuffle_each_iteration=False)

train_data = train_data.map(DataPreproc_map)

test_data = train_data.skip(3200)
valid_data = test_data.take(1000)
test_data = test_data.skip(1000)
train_data = train_data.take(3200)

for ex in train_data.take(3):
    print(ex[0].shape)

(7, 22050)
(2, 22050)
(1, 22050)


In [40]:
train_data = train_data.shuffle(BUFFER, reshuffle_each_iteration=False).padded_batch(BATCH_SIZE, padded_shapes=([None, None], [None]))
valid_data = valid_data.shuffle(BUFFER, reshuffle_each_iteration=False).padded_batch(BATCH_SIZE, padded_shapes=([None, None], [None]))
test_data = test_data.padded_batch(1, padded_shapes=([None, None], [None]))

In [37]:
#building LSTM 2 model

ex = train_data.take(1)
shape_1 = ex[0].shape[2]
output_shape = ex[1].shape[1]

bi_lstm_model = tf.keras.Sequential([
    layers.Bidirectional(layers.LSTM(32, name = 'lstm-1'), input_shape=(None, shape_1),
                                  name = 'bidir-lstm'),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(output_shape, activation = 'sigmoid')
])

bi_lstm_model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidir-lstm (Bidirectional)   (None, 64)                5653248   
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 86)                11094     
Total params: 5,672,662
Trainable params: 5,672,662
Non-trainable params: 0
_________________________________________________________________


In [38]:
#Compile and train the model

bi_lstm_model.compile(optimizer = tf.keras.optimizers.Adam(1e-3),
                      loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                      metrics = ['accuracy'])
history = bi_lstm_model.fit(train_data, 
                            validation_data = valid_data,
                            epochs = EPOCHS)


Epoch 1/10
107/107 [==============================] - 1051s 10s/step - loss: 5.5716 - accuracy: 0.0225 - val_loss: 5.6556 - val_accuracy: 0.0350
Epoch 2/10
107/107 [==============================] - 998s 9s/step - loss: 5.2922 - accuracy: 0.0919 - val_loss: 5.5316 - val_accuracy: 0.0400
Epoch 3/10
107/107 [==============================] - 1003s 9s/step - loss: 4.2209 - accuracy: 0.2419 - val_loss: 5.6851 - val_accuracy: 0.0450
Epoch 4/10
107/107 [==============================] - 1000s 9s/step - loss: 2.9891 - accuracy: 0.4647 - val_loss: 6.0829 - val_accuracy: 0.0490
Epoch 5/10
107/107 [==============================] - 1000s 9s/step - loss: 2.3166 - accuracy: 0.5797 - val_loss: 6.3532 - val_accuracy: 0.0530
Epoch 6/10
107/107 [==============================] - 986s 9s/step - loss: 1.9362 - accuracy: 0.6381 - val_loss: 6.6604 - val_accuracy: 0.0490
Epoch 7/10
107/107 [==============================] - 1032s 10s/step - loss: 1.7024 - accuracy: 0.6694 - val_loss: 6.9814 - val_accuracy:

ValueError: in user code:

    D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:941 test_function  *
        outputs = self.distribute_strategy.run(
    D:\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\Anaconda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:909 test_step  **
        y_pred = self(x, training=False)
    D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:886 __call__
        self.name)
    D:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\input_spec.py:168 assert_in

In [ ]:
# Evaluate on test data
test_results = bi_lstm_model.evaluate(test_data)
print('Test acc.: {:.2f}%'.format(test_results[1]*100))

In [8]:
def DataConstruct(foldername):
    DATA_simple = []
    DATA_spectogram = []
    LABELS = []
    filenames = os.listdir(foldername)
    df = pd.read_csv(foldername+'.csv', encoding='utf-8')
        
    for filename, labels in df.values:
        text = foldername + '/' + filename
        y, sr = librosa.load(text, mono = True)
    
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        data = np.array((np.mean(chroma_stft), np.mean(spec_cent), 
                        np.mean(spec_bw), np.mean(rolloff), 
                        np.mean(zcr), np.mean(mfcc)), dtype = np.float64)
        DATA.append(data)
        labels = tokenizer.tokenize(labels)
        label = np.zeros_like(label_names, dtype=np.int64)
        for lb in labels:
            label[label_names.index(lb)] = 1
        LABELS.append(label)
    
    scaler = MinMaxScaler()
    DATA = scaler.fit_transform(DATA)
    
    return DATA, LABELS

In [9]:
data, targets = DataConstruct('train_curated')

D:\Anaconda\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


In [17]:
train_data = tf.data.Dataset.from_tensor_slices((data, targets))

BATCH_SIZE = 3
BUFFER = 5000

train_data = train_data.shuffle(BUFFER, reshuffle_each_iteration=False)
test_data = train_data.skip(3200)
valid_data = test_data.take(1000).batch(BATCH_SIZE)
test_data = test_data.skip(1000).batch(BATCH_SIZE)
train_data = train_data.take(3200).batch(BATCH_SIZE)



In [18]:
# Building 3_model
# OverFIT

for ex in train_data.take(1):
    output_shape = ex[1].shape[1]
    input_shape = ex[0].shape[1]
    print('train example shapes | ', ex[0].shape)

inputs = keras.Input(shape=(input_shape))

x = layers.Dense(128, activation='relu', name='fc_1')(inputs)
x = layers.Dense(256, activation='relu', name='fc_2')(x)
x = layers.Dense(256, activation='relu', name='fc_3')(x)

y = layers.Dropout(0.2)(x)
z = layers.Dense(256)(y)

output = layers.Dense(output_shape, activation='sigmoid', name='output')(z)

model_1 = keras.Model(inputs = inputs, outputs = output)
model_1.summary()

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

hist_1 = model_1.fit(train_data, validation_data=valid_data, epochs = 150, shuffle=True)

test_res_1 = model_1.evaluate(test_data)

# OVERFIT
print('Test acc: ', test_res_1[1]*100, '%')

train example shapes |  (3, 6)
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
fc_1 (Dense)                 (None, 256)               1792      
_________________________________________________________________
fc_2 (Dense)                 (None, 528)               135696    
_________________________________________________________________
fc_3 (Dense)                 (None, 528)               279312    
_________________________________________________________________
fc_4 (Dense)                 (None, 528)               279312    
_________________________________________________________________
dropout_6 (Dropout)          (None, 528)               0         
_________________________________________________________________
dense_6 (Dense)             

1067/1067 [==============================] - 4s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 48/150
1067/1067 [==============================] - 4s 4ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 49/150
1067/1067 [==============================] - 4s 4ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 50/150
1067/1067 [==============================] - 4s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 51/150
1067/1067 [==============================] - 4s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 52/150
1067/1067 [==============================] - 3s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 53/150
1067/1067 [==============================] - 4s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy

1067/1067 [==============================] - 4s 4ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 104/150
1067/1067 [==============================] - 4s 4ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 105/150
1067/1067 [==============================] - 4s 4ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 106/150
1067/1067 [==============================] - 4s 4ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 107/150
1067/1067 [==============================] - 4s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 108/150
1067/1067 [==============================] - 3s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_accuracy: 0.0100
Epoch 109/150
1067/1067 [==============================] - 3s 3ms/step - loss: 5.6194 - accuracy: 0.0134 - val_loss: 5.5894 - val_ac